<a href="https://colab.research.google.com/github/DaheeKo/Deep-learning-study/blob/main/CODES/Chap11_Char_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chap 11. 다대다 RNN을 이용한 텍스트 생성**

다대다 RNN : 모든 시점의 입력에 대해서, 모든 시점에 대해서 출력

=> 품사 태깅, 개체명 인식 등에서 사용됨

1. 문자 단위 RNN (Char RNN): RNN의 입출력 단위가 문자 레벨(character level)

  예제) 문자 시퀀스 apple 입력 -> pple! 출력하도록 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 훈련 데이터 전처리
  # 입력 데이터와 레이블 데이터에 대해서 문자 집합(voabulary)을 만들기, 중복을 제거
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

  # 하이퍼파라미터 정의
input_size = vocab_size # 입력: 원-핫 벡터 사용할 것이므로 크기는 문자 집합의 크기 
hidden_size = 5
output_size = 5
learning_rate = 0.1

  # 고유한 정수 부여
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # enumerate: 자료형 입력 받아 인덱스 값 포함하는 객체로 리턴
print('char_to_index: ', char_to_index)

  # 정수로부터 문자를 얻을 수 있는 함수 구현 (결과를 문자 시퀀스로 받기 위해)
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print('index_to_char: ', index_to_char)

  # 데이터의 각 문자들을 정수로 맵핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print('x_data: ', x_data) # a, p, p, l, e에 해당
print('y_data: ', y_data) # p, p, l, e, !에 해당

  # 배치 차원 추가 <= nn.RNN()이 기본적으로 3차원 텐서를 입력 받기 때문
    # 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있음.
x_data = [x_data]
y_data = [y_data]
print('배치 차원 추가한 x_data: ', x_data)
print('배치 차원 추가한 y_data: ', y_data)

  # 입력 시퀀스의 각 문자들을 원-핫 벡터로 변경
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print('x_one_hot: ', x_one_hot)

  #입력 데이터와 레이블 데이터를 텐서로 바꿔줌
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

문자 집합의 크기 : 5
char_to_index:  {'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
index_to_char:  {0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}
x_data:  [1, 4, 4, 3, 2]
y_data:  [4, 4, 3, 2, 0]
배치 차원 추가한 x_data:  [[1, 4, 4, 3, 2]]
배치 차원 추가한 y_data:  [[4, 4, 3, 2, 0]]
x_one_hot:  [array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]
훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


In [ ]:
# 모델 구현하기 

  # 클래스로 모델 정의
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
  
net = Net(input_size, hidden_size, output_size)

outputs = net(X)
print('outputs.shape: ', outputs.shape) # 3차원 텐서: (배치차원, 시점, 출력의 크기)
      # 정확도 측정 시에는 모두 펼쳐서 계산: view 사용 -> 배치 차원과 시점 차원을 하나로
print('2차원 텐서로 변환: ', outputs.view(-1, input_size).shape) 
print('Y.shape: ', Y.shape) 
print('2차원 텐서로 변환: ', Y.view(-1).shape)
print()
  # 옵티마이저, 손실 함수 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

  # 100번의 에포크 학습
for i in range(100):
    optimizer.zero_grad() # Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문에 
                          # backpropagation을 하기전에 gradients를 zero로 만들어주고 시작
                          # 한 번 학습할 때마다 0으로 초기화
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

outputs.shape:  torch.Size([1, 5, 5])
2차원 텐서로 변환:  torch.Size([5, 5])
Y.shape:  torch.Size([1, 5])
2차원 텐서로 변환:  torch.Size([5])

0 loss:  1.5979819297790527 prediction:  [[4 4 4 3 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppplp
1 loss:  1.3819283246994019 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
2 loss:  1.268026351928711 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
3 loss:  1.15736722946167 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
4 loss:  1.0127224922180176 prediction:  [[4 4 2 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppee!
5 loss:  0.8821309208869934 prediction:  [[4 4 2 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppee!
6 loss:  0.7519644498825073 prediction:  [[4 4 3 3 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppll!
7 loss:  0.6107199788093567 prediction:  [[4 4 3 3 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppll!
8 loss:  0.51166170835495 predicti

2. 더 많은 데이터 문자 단위 RNN

In [ ]:
# 훈련 데이터 전처리
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

# 문자 집합 생성, 고유 정수 부여
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)} 
print("문자 집합: ", char_dic)
dic_size = len(char_dic) # 매 시점마다 들어갈 입력의 크기이기도 함
print('문자 집합의 크기 : {}'.format(dic_size))

# 하이퍼파라미터 설정
hidden_size = dic_size # 다른 값도 무방
sequence_length = 10  # 임의 숫자 지정, 앞서 만든 샘플을 10개 단위로 끊어서 샘플로 만들 것임
learning_rate = 0.1 # 학습률

# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence)-sequence_length):
  x_str = sentence[i: i + sequence_length]
  y_str = sentence[i+1: i + sequence_length + 1]
  #print(i, x_str, ' -> ', y_str)

  x_data.append([char_dic[c] for c in x_str]) # # x_str to index
  y_data.append([char_dic[c] for c in y_str])  # y_str to index

print('첫 번째 샘플의 입력 데이터: ', x_data[0])
print('첫 번째 샘플의 레이블 데이터: ', y_data[0]) # 한 칸씩 쉬프트 된 시퀀스 출력

# 입력 시퀀스에 대해 원-핫 인코딩 수행, 입력 및 레이블 데이터 텐서로 변환
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

print('원-핫 인코딩 된 결과: ', X[0])
print('레이블 데이터의 첫 번째 샘플: ', Y[0]) # f you want에 해당


문자 집합:  {'u': 0, 'a': 1, 't': 2, 's': 3, 'i': 4, 'l': 5, '.': 6, 'f': 7, 'd': 8, ',': 9, 'y': 10, 'e': 11, 'o': 12, 'n': 13, 'p': 14, 'k': 15, 'g': 16, "'": 17, 'h': 18, 'b': 19, 'm': 20, 'c': 21, ' ': 22, 'r': 23, 'w': 24}
문자 집합의 크기 : 25
첫 번째 샘플의 입력 데이터:  [4, 7, 22, 10, 12, 0, 22, 24, 1, 13]
첫 번째 샘플의 레이블 데이터:  [7, 22, 10, 12, 0, 22, 24, 1, 13, 2]
훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])
원-핫 인코딩 된 결과:  tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       

In [ ]:
# 모델 구현하기
 # 은닉층 2개 쌓음

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True) # num_layers 은닉층의 수
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개

 # 비용 함수와 옵티마이저 선언
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

outputs = net(X)
print('출력의 크기: ', outputs.shape) # 3차원 텐서 (배치차원, 시점, 출력의 크기)
print('2차원 텐서로 변환: ', outputs.view(-1, dic_size).shape) 
print('레이블 데이터의 크기: ', Y.shape) 
print('2차원 텐서로 변환: ', Y.view(-1).shape)
print()

 # 손실 함수와 옵티마이저 정의
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X) # 매 에포크마다 모델의 입력으로 사용

  loss = criterion(outputs.view(-1,  dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  results = outputs.argmax(dim=2) # results의 텐서 크기는 (170, 10)
  predict_str = ""

  for j, result in enumerate(results):
    if j==0: # 처음에는 예측 결과를 전부 가져오지만
      predict_str += ''.join([char_set[t] for t in result])
    else: # 그 다음에는 마지막 글자만 반복 추가
      predict_str += char_set[result[-1]]
      
  print(predict_str) # 처음에는 이상한 예측, 마지막 에포크에서는 꽤 정확한 문자 생성


출력의 크기:  torch.Size([170, 10, 25])
2차원 텐서로 변환:  torch.Size([1700, 25])
레이블 데이터의 크기:  torch.Size([170, 10])
2차원 텐서로 변환:  torch.Size([1700])

ggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggg
                                                                                                                                                                                   
    t t t t  t t t t  t t t t t t   t      t t t     t t t t tt t t t           t t t t t t t t t t   t t t t t t t t t t t    t t t t t    t t t t t t t t t t      t t t t t t t 
t ghtryhdrrirrrirriririrrraririrrrirrrririririhhiriiiirnirrrrhrrirhririririrrrarnrirrririrrrirorirrrrrrrrrrhroiririiarhoirhrrioorirhrrrrirrrirrriraiirrirhhrirriiriiidrirohrrirriir
toooaoooooooooooooooooooo ooooo oooooooooooooo oo ooo ooooooooooo o ooooo ooooooooo ooooooooooooooooooooooooooooooo o ooooooooooooooooooo  o

3. 단어 단위 RNN - 임베딩 사용

예제)  'Repeat is the best medicine for' 입력 
          -> 'is the best medicine for memory' 출력

In [ ]:
# 훈련 데이터 전처리

import torch
import torch.nn as nn
import torch.optim as optim

sentence = "Repeat is the best medicine for memory".split()

vocab = list(set(sentence))
print('단어 집합: ', vocab)

word2index = {tkn: i for i, tkn in enumerate(vocab,1)} # 고유한 정수 부여
word2index['<unk>'] = 0
print('word2index: ', word2index) # unk 토큰 추가된, 우리가 사용할 vocabulary

 # 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print('index2word: ', index2word)

def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

X, Y = build_data(sentence, word2index) # 입력 데이터, 레이블 데이터
print('입력 데이터: ', X) # Repeat is the best medicine for
print('레이블 데이터: ', Y) # Repeat is the best medicine for




단어 집합:  ['medicine', 'the', 'memory', 'is', 'Repeat', 'best', 'for']
word2index:  {'medicine': 1, 'the': 2, 'memory': 3, 'is': 4, 'Repeat': 5, 'best': 6, 'for': 7, '<unk>': 0}
index2word:  {1: 'medicine', 2: 'the', 3: 'memory', 4: 'is', 5: 'Repeat', 6: 'best', 7: 'for', 0: '<unk>'}
입력 데이터:  tensor([[5, 4, 2, 6, 1, 7]])
레이블 데이터:  tensor([[4, 2, 6, 1, 7, 3]])


In [ ]:
# 모델 구현 **임베딩 층을 추가**

class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.
    
    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

  # 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

  # 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
  # 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
  # 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

  # 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print('임의 예측, 출력 확인용: \n', output)
print('예측값의 크기: ', output.shape) # (시퀀스의 길이, 은닉층의 크기)

  # 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

 # 학습하기
for step in range(201):
    optimizer.zero_grad()    # 경사 초기화
    output = model(X)    # 순방향 전파
    loss = loss_function(output, Y.view(-1))    # 손실값 계산
    loss.backward()     # 역방향 전파
    optimizer.step()    # 매개변수 업데이트
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()


임의 예측, 출력 확인용: 
 tensor([[-0.4572, -0.2625, -0.2009, -0.0724, -0.3146, -0.1375,  0.0638, -0.0365],
        [-0.2208, -0.3214, -0.0030,  0.0147,  0.0791,  0.0096,  0.1051, -0.0884],
        [ 0.0016,  0.0666,  0.1854, -0.1536,  0.2216,  0.2227,  0.1517, -0.1729],
        [-0.1251, -0.0313, -0.0925,  0.0954, -0.1182, -0.0539,  0.1845, -0.3352],
        [ 0.1786, -0.1765,  0.4881,  0.0933,  0.7525,  0.1099,  0.1881, -0.0014],
        [-0.1199,  0.1940,  0.2107,  0.0678,  0.3825,  0.1778,  0.1012, -0.1582]],
       grad_fn=<ViewBackward>)
예측값의 크기:  torch.Size([6, 8])
[01/201] 2.1316 
Repeat best best Repeat best is is

[41/201] 1.5245 
Repeat memory the best medicine for memory

[81/201] 0.8926 
Repeat is the best medicine for memory

[121/201] 0.4468 
Repeat is the best medicine for memory

[161/201] 0.2281 
Repeat is the best medicine for memory

[201/201] 0.1291 
Repeat is the best medicine for memory

